In [ ]:
import math

def deviations(nlines, massmap, data, fit, vm):
    devfilename = "97deviations.dat"
    amassel = 0.511004 / 931.49386
    with open(devfilename, "w") as devfile:
        for i in range(nlines):
            devs = 0.0
            for j in range(polynom_degree):
                devs += fit[data[i]["spec"]][j] * math.pow(data[i]["fre"], j)
            dev = (vm[data[i]["massn"]] + amassel * (massmap[data[i]["massn"]][1] - data[i]["q"]) - data[i]["be"]) / data[i]["q"] - devs
            if i > 0 and (fit[data[i]["spec"]][0] != fit[data[i-1]["spec"]][0] or fit[data[i]["spec"]][1] != fit[data[i-1]["spec"]][1] or fit[data[i]["spec"]][2] != fit[data[i-1]["spec"]][2]):
                devfile.write("\n")
            devfile.write(f"{data[i]['massn']:3d} {massmap[data[i]['massn']][0]:3d} {massmap[data[i]['massn']][1]:2d} {massmap[data[i]['massn']][2]:2d} {massmap[data[i]['massn']][3]:1d} {vm[data[i]['massn']]:.8f} {dev*data[i]['q']:+.8f} {vm[data[i]['massn']]-dev*data[i]['q']:.8f} {data[i]['q']:d} {30.0*1.0e9/(data[i]['fre']+497.0)-60000000.0:.2f} {abs(dev*data[i]['q']):.8f}\n")


In [ ]:
import numpy as np

def Chi2Calculation(sfactor, nlines, massmap, data, fit, vm):
    amassel = 0.511004/931.49386
    sum = 0.0
    chi2 = 0.0
    for i in range(nlines):
        substr = 0.0
        for j in range(polynom_degree):
            substr += fit[data[i].spec][j]*pow(data[i].fre,j)
        sum = pow(((vm[data[i].massn]+amassel*(massmap[data[i].massn][1]-data[i].q)-data[i].be)/data[i].q-substr), 2.0)
        if(sfactorswitch == 0):
            efre = 0.0
            for k in range(1, polynom_degree):
                efre += k*fit[data[i].spec][k]*pow(data[i].fre,k-1)*data[i].efre
            efre = abs(efre)
        if(sfactorswitch == 1):
            efre = sfactor*data[i].efre
        sum = sum/pow(efre, 2.0)
        chi2 += sum
        sum = 0.0
    chi2 /= nlines
    chi2 = np.sqrt(chi2)
    return chi2

def SFactorCalculation(nlines, massmap, data, fit, vm):
    amassel = 0.511004/931.49386
    sum = 0.0
    sfactor = 0.0
    for i in range(nlines):
        substr = 0.0
        for j in range(polynom_degree):
            substr += fit[data[i].spec][j]*pow(data[i].fre,j)
        if(sfactorswitch == 0):
            efre = 0.0
            for k in range(1, polynom_degree):
                efre += k*fit[data[i].spec][k]*pow(data[i].fre,k-1)*data[i].efre
            efre = abs(efre)
        if(sfactorswitch == 1):
            efre = data[i].efre
        sum = ((vm[data[i].massn]+amassel*(massmap[data[i].massn][1]-data[i].q)-data[i].be)/data[i].q-substr)*((vm[data[i].massn]+amassel*(massmap[data[i].massn][1]-data[i].q)-data[i].be)/data[i].q-substr)/(efre*efre)
        sfactor += sum
        sum = 0.0
    sfactor /= nlines
    sfactor = np.sqrt(sfactor)
    return sfactor

In [ ]:
import numpy as np

polynom_degree = 5
amassel = 0.511004 / 931.49386
masstab_scale = 1.0

def CalculateFits(sfactor, nmasses, nspectra, nlines, data, spdata, massmap, fit, vm):
    size_full = polynom_degree
    size_this = size_full
    stpos = 0 # pointer to the data array
    fipos = 0
    fitcoef = np.zeros(size_full)
    ma = np.zeros((size_full, size_full))
    mb = np.zeros((size_full, size_full))

    for i in range(nspectra): # main array
        stpos = fipos
        for j in range(stpos, nlines):
            if data[j].spec == i:
                pass
            if data[j].spec < i:
                print("Strange")
            if data[j].spec > i:
                fipos = j
                break
        if i == nspectra-1:
            fipos = nlines
        if stpos == fipos:
            break
        size_this = spdata[i].poldegree + 1
        ma[:,:] = 0.0
        for j in range(stpos, fipos):
            if sfactor == 0:
                efrej = 0.0
                for l in range(1, size_this):
                    efrej += l * fit.Matrix[data[j].spec][l] * pow(data[j].fre, l-1) * data[j].efre
                efrej = abs(efrej)
            elif sfactor == 1:
                efrej = sfactor * data[j].efre
            for l in range(size_this):
                for g in range(size_this):
                    ma[l][g] += masstab_scale * pow(data[j].fre, l+g) / (efrej*efrej)
        tma = np.linalg.inv(ma[:size_this,:size_this])
        if np.abs(CheckInversion(size_this, size_full, ma, tma) - size_this) > 1e-4:
            print(" *!*!* CHI2: S:%17.15lf A:%17.15lf\n", CheckInversion(size_this, size_full, ma, tma), size_this/CheckInversion(size_this, size_full, ma, tma))
        ma[:,:] = np.dot(ma, masstab_scale)
        fitcoef[:] = fit.Matrix[data[stpos].spec,:size_this]
        fit.Matrix[i,:] = 0.0
        for j in range(stpos, fipos):
            if sfactor == 0:
                efrej = 0.0
                for l in range(1, size_this):
                    efrej += l * fitcoef[l] * pow(data[j].fre, l-1) * data[j].efre
                efrej = abs(efrej)
            elif sfactor == 1:
                efrej = sfactor * data[j].efre
            pj = 1.0 / (efrej*efrej)
            x = data[j].massn
            for l in range(size_this):
                for g in range(size_this):
                    fit.Matrix[i][l] += ma[l][g] * pj * pow(data[j].fre, g) * ((vm[x] + amassel * (massmap[x][1] - data[j].q) - data[j].be) / data[j].q)
    import ctypes
    ctypes.free(fitcoef)
    for i in range(nspectra):
        kill_matrix(m[i])
    free(m)
    for i in range(nspectra):
        kill_matrix(mf[i])
    free(mf)

# shubina

In [ ]:
import numpy as np
from numpy.linalg import inv

def fill_matrix_A(mass, freq, delta_sq, charge, benergy, item, identification, S, max_pol_degree):
    A = []
    pol_degree = []
    for s in range(S):
        if len(mass[s]) < 10:
            pol_degree.append(min(3, max_pol_degree))
        elif len(mass[s]) < 8:
            pol_degree.append(min(2, max_pol_degree))
        elif len(mass[s]) < 5:
            pol_degree.append(1)
        else:
            pol_degree.append(max_pol_degree)
        
        data = np.zeros((pol_degree[s]+1)*(pol_degree[s]+1))
        for p in range(pol_degree[s]+1):
            for q in range(pol_degree[s]+1):
                tmp = 0
                for in_ in range(len(mass[s])):
                    tmp += 1.0/delta_sq[s][in_]*freq[s][in_]**(p+q)
                data[p*(pol_degree[s]+1)+q] = tmp
        A.append(data.reshape(pol_degree[s]+1, pol_degree[s]+1))

    AI = []
    for s in range(S):
        AI.append(inv(A[s]))

    return AI, pol_degree

def fill_matrix_W(mass, freq, delta_sq, charge, benergy, item, identification, S, I, AI, pol_degree):
    W = np.zeros((I, I))
    for i in range(I):
        for j in range(I):
            W[i,j] = 0

    w = np.zeros(I)
    for i in range(I):
        B = 0
        w[i] = 0
        fAdded = False
        for s in range(S):
            count = 0
            f = 0
            for n in range(len(freq[s])):
                if item[s][n] == identification[i]:
                    fr_H = np.zeros(pol_degree[s]+1)
                    for p in range(pol_degree[s]+1):
                        fr_H[p] = freq[s][n]**p
                    fr_H2 = np.matmul(fr_H, AI[s])
                    for j in range(I):
                        for m in range(len(freq[s])):
                            if item[s][m] == identification[j]:
                                fr_V = np.zeros(pol_degree[s]+1)
                                for p in range(pol_degree[s]+1):
                                    fr_V[p] = freq[s][m]**p
                                nm = np.matmul(fr_H2, fr_V)[0]
                                if n == m:
                                    W[i,j] += 1/(delta_sq[s][n]*charge[s][n]**2)
                                    B -= 1/(delta_sq[s][n]*charge[s][n]**2)*benergy[s][n]
                                W[i,j] -= 1/(delta_sq[s][n]*charge[s][n]*delta_sq[s][m]*charge[s][m])*nm
                                B += 1/(delta_sq[s][n]*charge[s][n]*delta_sq[s][m]*charge[s][m])*nm*benergy[s][m]
        w[i] = B
    return W, w

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def set_delta_sq(iter, mass, charge, freq, dfreq, identification, item, benergy, pol_degree, AI, masses):
    S = len(mass)
    I = len(identification)

    delta_sq = []
    for s in range(S):
        delta_in = []
        if iter == 0:
            mtoq = np.array(mass[s]) / np.array(charge[s])
            popt, _ = curve_fit(lambda x, a, b: a * x + b, freq[s], mtoq)
            par1 = popt[0]
            for in_ in range(len(mass[s])):
                delta_in.append((par1 * dfreq[s][in_]) ** 2)
        else:
            for in_ in range(len(mass[s])):
                col = np.zeros((pol_degree[s] + 1, 1))
                for k in range(pol_degree[s] + 1):
                    st = 0
                    for jm in range(len(mass[s])):
                        for id_ in range(I):
                            if identification[id_] == item[s][jm]:
                                st += 1 / delta_sq[s][jm] * (masses[id_, 0] + benergy[s][jm]) / charge[s][jm] * freq[s][jm] ** k
                                break
                    col[k, 0] = st
                as_ = np.dot(AI[s], col)
                dtmp = 0
                for k in range(1, pol_degree[s] + 1):
                    dtmp += k * as_[k, 0] * freq[s][in_] ** (k - 1) * dfreq[s][in_]
                mass_in_sp = 0
                for k in range(pol_degree[s] + 1):
                    mass_in_sp += as_[k, 0] * freq[s][in_] ** k
                delta_in.append(dtmp ** 2)
        delta_sq.append(delta_in)
    return delta_sq


In [ ]:
def compare_iter(masses, masses_prev):
    fContinue = False
    for i in range(I):
        diff = (masses[i, 0] - masses_prev[i, 0]) * 9.31494
        # print(f"{diff:5.0f}", end=" ")
        if diff > 1:
            fContinue = True
    # print()
    return fContinue


def print_results():
    print("Identity     is ref(0) \t mass in u \t\t ame_mass in u   diff \n")
    for i in range(I):
        id_name = identification[i]
        # print(identification[i], id_name)
        z = int(id_name[0:2])
        a = int(id_name[2:5])
        # q = int(id_name[5:7])
        q = z
        is_ = int(id_name[7:9])
        with open(nuclide_table, "r") as table:
            for line in table:
                tA, tZ, tQ, tEl, ti, table_mass, tmdm, tex, tE, tEde, tThl = line.split()
                tA, tZ, tQ, ti = int(tA), int(tZ), int(tQ), int(ti)
                if tA == a and tZ == z and tQ == z and ti == is_:
                    print(
                        f"{z:2d} {a:3d} {q:2d} {is_:1d} \t {ref[i]:1d} \t {masses[i, 0]/100000:10.6f}   {masses[i, 1]*amu/100000:6.2f} \t {table_mass*z:10.6f} {amu*(masses[i, 0]/100000 - table_mass*z):10.2f}"
                    )
                    break


In [ ]:
def change_refs(id_change):
    global ref_backup
    ref_backup = ref.copy()
    counter = 0
    for i in range(len(ref)):
        if ref[i] == 0:
            if counter == id_change:
                ref[i] = 1
                print(identification[i], "switched off")
                break
            counter += 1

def change_refs_back():
    global ref
    ref = ref_backup.copy()

def backup_vectors():
    global mass_backup, dmass_backup, freq_backup, dfreq_backup, item_backup, charge_backup, benergy_backup, identification_backup, ref_backup, I_b
    mass_backup = mass.copy()
    dmass_backup = dmass.copy()
    freq_backup = freq.copy()
    dfreq_backup = dfreq.copy()
    item_backup = item.copy()
    charge_backup = charge.copy()
    benergy_backup = benergy.copy()
    identification_backup = identification.copy()
    ref_backup = ref.copy()
    I_b = I

def check_extrapolation():
    fContinue = True
    while fContinue == True:
        fContinue = False
        for i in range(len(freq)):
            for k in range(I):
                if item[i][0] == identification[k]:
                    if ref[k] == 1:
                        freq[i].pop(0)
                        dfreq[i].pop(0)
                        charge[i].pop(0)
                        benergy[i].pop(0)
                        mass[i].pop(0)
                        dmass[i].pop(0)
                        item[i].pop(0)
                        # print("erased first in ", i)
                        fContinue = True
                    break
        i = 0
    while fContinue == True:
        fContinue = False
        for i in range(len(freq)):
            for k in range(I):
                if item[i][-1] == identification[k]:
                    if ref[k] == 1:
                        freq[i].pop()
                        dfreq[i].pop()
                        charge[i].pop()
                        benergy[i].pop()
                        mass[i].pop()
                        dmass[i].pop()
                        item[i].pop()
                        # print("erased last in ", i)
                        fContinue = True
                    break
        i = 0
    while i < I:
        found = False
        for s in range(len(freq)):
            for n in range(len(freq[s])):
                if item[s][n] == identification[i]:
                    found = True
        if found == False:
            identification.pop(i)
            ref.pop(i)
            I -= 1
            print(i, "deleted")
        else:
            i += 1

def restore_vectors():
    global mass, dmass, freq, dfreq, item, charge, benergy, identification, ref, I
    mass = mass_backup.copy()
    dmass = dmass_backup.copy()
    freq = freq_backup.copy()
    dfreq = dfreq_backup.copy()
    item = item_backup.copy()
    charge = charge_backup.copy()
    benergy = benergy_backup.copy()
    identification = identification_backup.copy()
    ref = ref_backup.copy()
    I = I_b

# Shubina july 2023 (splines?)

In [ ]:
import numpy as np

def find_mass_in_table(a, z, q, is):
        with open(nuclide_table, 'r') as table:
            for line in table:
                tA, tZ, tQ, tEl, ti, table_mass, tmdm, tex, tE, tEde, tThl = map(float, line.split())
                if tA == a and tZ == z and tQ == q and ti == is:
                    return True, table_mass, tmdm / 1000000 / q  # Convert to desired units
        return False, None, None
    
def fill_vectors(nuclide_table, db_name):
    mass_ref = []
    mdmass_ref = []
    item_ref = []
    freq_ref = []
    dfreq_ref = []
    
    item_un = []
    freq_un = []
    dfreq_un = []
    identification = []

    with open(db_name, 'r') as infile:
        while True:
            line = infile.readline().strip()
            if not line:
                break

            f = float(line)
            if f == 0:
                # Add data to reference vectors and clear temporary vectors
                mass_ref.append(np.array(mass_ref_sp))
                mdmass_ref.append(np.array(dmass_ref_sp))
                item_ref.append(np.array(item_ref_sp))
                freq_ref.append(np.array(freq_ref_sp))
                dfreq_ref.append(np.array(dfreq_ref_sp))
                
                mass_ref_sp.clear()
                dmass_ref_sp.clear()
                item_ref_sp.clear()
                freq_ref_sp.clear()
                dfreq_ref_sp.clear()
            else:
                errorf, z, a, q, is, que = map(float, infile.readline().split())
                found_in_table, table_mass, tmdm = find_mass_in_table(a, z, q, is)

                if found_in_table:
                    idn = f"{int(z):02d}{int(a):03d}{int(q):02d}{int(is):1d}{int(que):05d}"
                    id = idn.ljust(15)[:15]

                    if que == 0 and tex == 0 and tmdm < 50:
                        mass_ref_sp.append(table_mass)
                        dmass_ref_sp.append(tmdm)
                        freq_ref_sp.append(f)
                        dfreq_ref_sp.append(errorf)
                        item_ref_sp.append(id)
                    else:
                        freq_un.append(f)
                        dfreq_un.append(errorf)
                        item_un.append(id)

                    if id not in identification:
                        for n, nline in enumerate(identification):
                            nl = int(nline[8:13])
                            if int(id[8:13]) < nl:
                                identification.insert(n, id)
                                break
                        else:
                            identification.append(id)

    outname = db_name + ".masses"
    with open(outname, 'w'):
        pass

    return mass_ref, mdmass_ref, item_ref, freq_ref, dfreq_ref, item_un, freq_un, dfreq_un


# Usage example:
nuclide_table = "path/to/nuclide_table.txt"
db_name = "path/to/db_name.txt"
mass_ref, mdmass_ref, item_ref, freq_ref, dfreq_ref, item_un, freq_un, dfreq_un = fill_vectors(nuclide_table, db_name)


In [ ]:
from collections import defaultdict

def change_vectors(n):
    number = n
    counter = 0
    identification_indices = defaultdict(list)

    # Group the indices of items with the same identification
    for i, item_list in enumerate(item_ref):
        for j, item in enumerate(item_list):
            identification_indices[item].append((i, j))

    # Process each identification and update item_ref and related lists
    for identification_id, id_indices in identification_indices.items():
        for i, j in id_indices:
            if counter == number:
                item_un[i].append(item_ref[i][j])
                freq_un[i].append(freq_ref[i][j])
                dfreq_un[i].append(dfreq_ref[i][j])

                del item_ref[i][j]
                del freq_ref[i][j]
                del dfreq_ref[i][j]
                del mass_ref[i][j]
                del mdmass_ref[i][j]
                break  # Exit the inner loop after erasing the elements

        counter += 1

# Note: mass_ref, mdmass_ref, item_ref, freq_ref, dfreq_ref, item_un, freq_un, and dfreq_un
# should be defined and populated with data before calling the change_vectors function.


In [ ]:
def true_ref():
    diff_table = [0] * len(identification)
    diff_err_table = [0] * len(identification)
    nref = 0

    for id, ident in enumerate(identification):
        ref = 0
        for i, item_list in enumerate(item_ref):
            if ident in item_list:
                ref = 1

        if ref != 0 and nref < hstat.GetYaxis().GetNbins():
            diff_table[id] = hstat.GetBinContent(id + 1, nref + 1)
            diff_err_table[id] = hstat.GetBinError(id + 1, nref + 1)
            nref += 1


In [ ]:
import numpy as np
from scipy.interpolate import interp1d

def average_f(n):
    iteration = n + 1
    hstat.SetBins(len(identification), 0, len(identification), n + 1, 0, n + 1)

    table_graph_f = []
    table_graph_m = []
    table_graph_dm = []

    reference_f = []
    reference_df = []
    reference_mass = []
    reference_mdmass = []
    reference_id = []

    unknown_f = []
    unknown_df = []
    unknown_id = []
    unknown_mass = []
    unknown_dmass = []
    unknown_diff = []

    unknown_counter = []
    ident = ""

    # Averaging
    for id, ident in enumerate(identification):
        hf = np.array([])

        counter = 0
        mass, dm = 0, 0

        for i, item_list in enumerate(item_ref):
            for j, item in enumerate(item_list):
                if item == ident:
                    counter += 1
                    hf = np.append(hf, freq_ref[i][j])
                    mass, dm = mass_ref[i][j], mdmass_ref[i][j]

        if counter != 0:
            a0 = np.polyfit(range(1, counter + 1), hf, 0)[0]
            a0er = 0  # Error term not provided in the original code

            reference_f.append(a0)
            reference_df.append(a0er)
            reference_mass.append(mass)
            reference_mdmass.append(mass + dm + a0er * coefficient)
            reference_id.append(ident)

        else:
            for i, item_list in enumerate(item_un):
                for j, item in enumerate(item_list):
                    if item == ident:
                        counter += 1
                        hf = np.append(hf, freq_un[i][j])

            if counter != 0:
                a0 = np.polyfit(range(1, counter + 1), hf, 0)[0]
                a0er = 0  # Error term not provided in the original code

                unknown_f.append(a0)
                unknown_df.append(a0er)
                unknown_id.append(ident)
                unknown_counter.append(counter)

    # Linear spline interpolation for evaluation
    lin_sp_1 = interp1d(reference_f, reference_mass, kind='linear', fill_value="extrapolate")
    lin_sp_2 = interp1d(reference_f, reference_mdmass, kind='linear', fill_value="extrapolate")

    for i, u_f in enumerate(unknown_f):
        min_dist = min(abs(u_f - r_f) for r_f in reference_f)
        if reference_f[0] < u_f < reference_f[-1] and min_dist < 15000 and 200000 > u_f > 77000:
            mass = lin_sp_1(u_f)
            dm = lin_sp_2(u_f) - lin_sp_1(u_f)
            unknown_mass.append(mass)

            identity = identification.index(unknown_id[i])

            with open(outname, "a+") as out:
                id_name = unknown_id[i]
                z = int(id_name[0:2])
                a = int(id_name[2:5])
                q = int(id_name[5:7])
                is_val = int(id_name[7])
                unknown_dmass.append((abs(dm) + unknown_df[i] * coefficient) * q * amu)
                out.write(f"{unknown_counter[i]:3d} {z:2d} {a:3d} {q:2d} {is_val:1d} 1.0 {mass:.9f} 0.0 {abs(dm) + unknown_df[i] * coefficient:7.1f} {abs(dm) + unknown_df[i] * coefficient:5.1f}\n")
                unknown_diff.append((mass - reference_mass[identity]) * q * amu)
                table_graph_f.append(u_f)
                table_graph_m.append(mass)
                table_graph_dm.append(dm / q / amu)
                hstat.SetBinContent(identity + 1, iteration, (mass - reference_mass[identity]) * q * amu)
                hstat.SetBinError(identity + 1, iteration, (abs(dm) + unknown_df[i] * coefficient) * q * amu)
        else:
            if u_f >= 200000 or u_f <= 77000:
                print(f"{u_f} More than cutoff 200 kHz")
            else:
                if min_dist > 15000:
                    print(f"{u_f} : min distance to next point more than 20 kHz")
                else:
                    print(f"Frequency {u_f} Hz out of reference range!")

            unknown_mass.append(0)
            unknown_dmass.append(0)
            unknown_diff.append(0)
            
    # Write newline to the file
    with open(outname, "a+") as out:
        out.write("\n")

    # Create TGraph objects
    table_graph = TGraph(len(table_graph_f), np.array(table_graph_f), np.array(table_graph_m))
    table_graph_dm = TGraph(len(table_graph_f), np.array(table_graph_f), np.array(table_graph_dm))

    # Save TGraph objects to a file
    with open(outname, "a+") as out:
        for i in range(len(table_graph_f)):
            out.write(f"{table_graph_f[i]:.5f} {table_graph_m[i]:.9f} {table_graph_dm[i]:.5f}\n")

    # Cleanup (delete objects)
    lin_sp_1.Delete()
    lin_sp_2.Delete()
    table_graph.Delete()
    table_graph_dm.Delete()


In [ ]:
def return_vectors():
    global mass_ref, mdmass_ref, item_ref, freq_ref, dfreq_ref, item_un, freq_un, dfreq_un
    mass_ref = list(mass_ref_copy)
    mdmass_ref = list(mdmass_ref_copy)
    item_ref = list(item_ref_copy)
    freq_ref = list(freq_ref_copy)
    dfreq_ref = list(dfreq_ref_copy)

    item_un = list(item_un_copy)
    freq_un = list(freq_un_copy)
    dfreq_un = list(dfreq_un_copy)


In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

# Assumed defined global variables and functions
db_name = ""
outname = ""
identification = []
unknown_f = []
unknown_diff = []
unknown_dmass = []
mass_ref = []
mdmass_ref = []
item_ref = []
freq_ref = []
dfreq_ref = []
item_un = []
freq_un = []
dfreq_un = []
diff_table = []
diff_err_table = []
coefficient = 0.0
hstat = None

# Functions (not shown here) - fill_vectors, average_f, change_vectors, return_vectors, true_ref

def main(argv):
    global db_name, outname, identification, unknown_f, unknown_diff, unknown_dmass, \
        mass_ref, mdmass_ref, item_ref, freq_ref, dfreq_ref, item_un, freq_un, dfreq_un, \
        diff_table, diff_err_table, coefficient, hstat

    if len(argv) > 1:
        db_name = argv[1]
    else:
        print("1st. argument = database name. db contains f df z a q is que. For example: 44464.00 3.78 77 195 75 0 0. And a '0' on new line to indicate the end of the spectrum.")
        return 0

    fill_vectors()
    average_f(0)
    upper_limit = len(identification) - len(unknown_f)
    for i in range(1, upper_limit - 1):
        change_vectors(i)
        average_f(i)
        return_vectors()
    true_ref()
    average_f(upper_limit - 1)

    # Rest of the code for plotting using matplotlib and numpy
    gr = plt.figure(figsize=(14, 8))
    ax1 = gr.add_subplot(121)
    ax2 = gr.add_subplot(122)

    ax1.grid(True, which='both', linestyle='dotted', linewidth=0.5)
    ax1.bar(range(1, len(identification) + 1), diff_table, color='lightgray', label='Switched off references')
    ax1.bar(range(1, len(identification) + 1), unknown_diff, color='lightblue', label='Unknowns', bottom=diff_table)
    ax1.legend(loc='upper right')
    ax1.set_title('Deviation')
    ax1.set_xlabel('Identity')
    ax1.set_ylabel('Deviation')

    ax2.errorbar(reference_f, np.array(reference_mass) - a1 * np.array(reference_f) - a0,
                 yerr=np.array(reference_df) * coefficient + np.array(reference_mdmass) - np.array(reference_mass),
                 fmt='o', label='References', color='b')
    ax2.errorbar(unknown_f, np.array(unknown_mass) - a1 * np.array(unknown_f) - a0,
                 yerr=np.array(unknown_df) * coefficient, fmt='o', label='Unknowns', color='g')
    ax2.errorbar(table_graph_f, np.array(table_graph_m) - a1 * np.array(table_graph_f) - a0,
                 yerr=np.array(table_graph_dm), fmt='o', label='Table Value for Unknown', color='r')
    ax2.legend(loc='upper right')
    ax2.set_title('m/q - pol1')
    ax2.set_xlabel('Frequency')
    ax2.set_ylabel('Mass')

    plt.suptitle('Mass Analysis')
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    plt.show()

if __name__ == "__main__":
    main(sys.argv)


# SHubina CMM

In [ ]:
import numpy as np

# Assumed defined global variables
nuclide_table = ""  # The path to the nuclide table file
db_name = ""        # The path to the data-file
kDivider = 0.0      # Some value for kDivider (not defined in the code)
kDivider2 = 0.0     # Some value for kDivider2 (not defined in the code)
kCoef = 0.0         # Some value for kCoef (not defined in the code)
amu = 0.0           # Some value for amu (not defined in the code)
m_e = 0.0           # Some value for m_e (not defined in the code)
ebinding_table = "" # The path to the ebinding table file

# Assumed defined global vectors
identification = []
ref = []
mass = []
dmass = []
item = []
freq = []
dfreq = []
charge = []
benergy = []
refs_values = []
refs_unc_values = []

def fill_vectors():
    # Temporary vectors for one spectrum
    mass_sp = []
    dmass_sp = []
    item_sp = []
    freq_sp = []
    dfreq_sp = []
    charge_sp = []
    benergy_sp = []

    # Open nuclide table
    table = open(nuclide_table, 'r')
    nline_table = 0

    # Open data-file
    infile = open(db_name, 'r')
    read = True
    region = 0
    previous_region = 0

    for line in infile:
        if read:
            f = float(line.strip())
        if not line.strip():
            break

        previous_region = region
        if 0 < f <= kDivider:
            region = 1
        elif kDivider < f <= kDivider2:
            region = 2
        else:
            region = 3

        if f != 0 and (region == 1 or region == previous_region or len(freq_sp) == 0):
            read = True
            errorf, z, a, q, is, que = map(float, infile.readline().split())
            found_in_table = False

            for table_line in table:
                tA, tZ, tQ, tEl, ti, table_mass, tmdm, tex, tE, tEde, tThl = map(float, table_line.split())
                nline_table += 1

                if tA == a and tZ == z and tQ == q and ti == is:
                    found_in_table = True
                    idn = "{:02d}{:03d}{:01d}".format(int(z), int(a), int(is))
                    idd = int(idn)
                    mass_sp.append(table_mass * q * kCoef)
                    dmass_sp.append((tmdm * kCoef / 1E6))
                    freq_sp.append(f / kCoef)
                    dfreq_sp.append(errorf / kCoef)
                    item_sp.append(idd)
                    charge_sp.append(q)

                    if z - q != 0:
                        bz = None
                        be = [0] * 4

                        with open(ebinding_table, 'r') as inbind:
                            for bind_line in inbind:
                                bz, be[0], be[1], be[2], be[3] = map(int, bind_line.split())
                                if bz == z:
                                    benergy_sp.append(((z - q) * m_e - be[z - q - 1] / 1000) / amu * 100000)
                                    break

                        if bz != z:
                            benergy_sp.append(0)
                    else:
                        benergy_sp.append(0)

                    fIdentified = False
                    for l in range(len(identification)):
                        if idd == identification[l]:
                            fIdentified = True
                            break

                    if tmdm > 10:
                        que = 1

                    if not fIdentified:
                        found = False
                        for n in range(len(identification)):
                            if idd < identification[n]:
                                identification.insert(n, idd)
                                ref.insert(n, que)
                                found = True
                                break

                        if not found:
                            identification.append(idd)
                            ref.append(que)

                    break

            if not found_in_table:
                print("{:.2f} {:d} {:d} {:d} {:d} was not found. "
                      "This can happen in 2 cases: m/q is not a monotonic function of frequency "
                      "(wrong identification or 2 points with the same identification) or mass does not exist in the table."
                      .format(f, int(z), int(a), int(q), int(is)))

                table = open(nuclide_table, 'r')  # Close and reopen table of nuclides to start searching from the beginning with the next mass
                nline_table = 0
        else:
            if f != 0:
                read = False

            if len(mass_sp) > 5:
                mass.append(mass_sp)
                dmass.append(dmass_sp)
                item.append(item_sp)
                freq.append(freq_sp)
                dfreq.append(dfreq_sp)
                charge.append(charge_sp)
                benergy.append(benergy_sp)

            mass_sp.clear()
            dmass_sp.clear()
            item_sp.clear()
            freq_sp.clear()
            dfreq_sp.clear()
            charge_sp.clear()
            benergy_sp.clear()

            table = open(nuclide_table, 'r')
            nline_table = 0

    infile.close()
    table.close()

    S = len(mass)  # Number of spectra
    I = len(identification)  # Number of identities
    nRefs = ref.count(0)
    R = nRefs
    refs_values.resize(I)
    refs_unc_values.resize(I)

    return


In [ ]:
import numpy as np

# Assumed defined global variables
S = 0                   # Number of spectra (You should define this)
max_pol_degree = 3       # Maximum polynomial degree (You should define this)

# Assumed defined global vectors
mass = []               # List of lists containing masses for each spectrum
freq = []               # List of lists containing frequencies for each spectrum
delta_sq = []           # List of lists containing delta_sq for each spectrum
pol_degree = []         # List to store polynomial degrees for each spectrum

# Function to fill matrix A
def fill_matrix_A():
    global S

    A = []       # A is a list of matrices A_s
    AI = []      # AI is a list of inverted matrices
    A.clear()
    A = [np.zeros((0, 0)) for _ in range(S)]  # Resize A for each spectrum
    pol_degree.clear()
    pol_degree = [0] * S  # Resize pol_degree for each spectrum

    for s in range(S):  # Separate A for each spectrum
        if len(mass[s]) < 10:
            pol_degree[s] = min(3, max_pol_degree)
        else:
            pol_degree[s] = max_pol_degree

        if len(mass[s]) < 8:
            pol_degree[s] = min(2, max_pol_degree)

        if len(mass[s]) < 5:
            pol_degree[s] = 1

        data = np.zeros((pol_degree[s] + 1) * (pol_degree[s] + 1))
        A[s] = np.zeros((pol_degree[s] + 1, pol_degree[s] + 1))

        for p in range(pol_degree[s] + 1):
            for q in range(pol_degree[s] + 1):
                tmp = 0
                for inx in range(len(mass[s])):
                    tmp += 1.0 / delta_sq[s][inx] * (freq[s][inx] ** (p + q))
                data[p * (pol_degree[s] + 1) + q] = tmp

        A[s] = np.matrix(data).reshape((pol_degree[s] + 1, pol_degree[s] + 1))

    AI = [np.linalg.inv(A[s]) for s in range(S)]

    return AI

# Call the function to fill matrix A
AI_matrices = fill_matrix_A()


In [ ]:
import numpy as np

# Assumed defined global variables
I = 0                # Number of identities (You should define this)
S = 0                # Number of spectra (You should define this)
max_pol_degree = 3   # Maximum polynomial degree (You should define this)

# Assumed defined global vectors
freq = []            # List of lists containing frequencies for each spectrum
delta_sq = []        # List of lists containing delta_sq for each spectrum
item = []            # List of lists containing identities for each spectrum
identification = []  # List containing the identification values
ref = []             # List containing reference values
pol_degree = []      # List containing polynomial degrees for each spectrum
benergy = []         # List of lists containing benergy for each spectrum
charge = []          # List of lists containing charge for each spectrum
masses_prev = np.zeros((I, 1))
masses = np.zeros((I, 2))
masses_new = np.zeros((I, 2))

# Function to fill matrix W
def fill_matrix_W():
    global I, S, max_pol_degree, masses_prev, masses, masses_new

    W = np.zeros((I, I))
    w = [0] * I

    for i in range(I):
        B = 0
        w[i] = 0
        fAdded = False

        for s in range(S):
            count = 0
            f = 0

            for n in range(len(freq[s])):
                if item[s][n] == identification[i]:
                    fr_H = np.zeros((1, pol_degree[s] + 1))
                    for p in range(pol_degree[s] + 1):
                        fr_H[0, p] = freq[s][n] ** p

                    fr_H2 = np.dot(fr_H, AI_matrices[s])
                    
                    for j in range(I):
                        for m in range(len(freq[s])):
                            if item[s][m] == identification[j]:
                                fr_V = np.zeros((pol_degree[s] + 1, 1))
                                for p in range(pol_degree[s] + 1):
                                    fr_V[p, 0] = freq[s][m] ** p
                                nm = np.dot(fr_H2, fr_V)[0, 0]

                                if n == m:
                                    W[i, j] += 1 / (delta_sq[s][n] * charge[s][n] ** 2)
                                    B -= 1 / (delta_sq[s][n] * charge[s][n] ** 2) * benergy[s][n]

                                W[i, j] -= 1 / (delta_sq[s][n] * charge[s][n] * delta_sq[s][m] * charge[s][m]) * nm
                                B += 1 / (delta_sq[s][n] * charge[s][n] * delta_sq[s][m] * charge[s][m]) * nm * benergy[s][m]

                    if ref[i] == 0 and not fAdded:
                        W[i, i] += 1 / (dmass[s][n] ** 2)
                        w[i] += (mass[s][n] - benergy[s][n]) / (dmass[s][n] ** 2)
                        fAdded = True

                    count += 1

        w[i] += B

    Wi = np.copy(W)
    Wi_inv = np.linalg.inv(Wi)

    for i in range(I):
        masses_new[i, 0] = np.dot(Wi_inv[i], w)
        masses_new[i, 1] = np.sqrt(Wi_inv[i, i])

    masses_prev = np.copy(masses)
    masses = np.copy(masses_new)

# Call the function to fill matrix W
fill_matrix_W()


In [ ]:
import numpy as np
from scipy.optimize import curve_fit

# Assumed defined global variables
I = 0                # Number of identities (You should define this)
S = 0                # Number of spectra (You should define this)
max_pol_degree = 3   # Maximum polynomial degree (You should define this)

# Assumed defined global vectors
mass = []            # List of lists containing mass values for each spectrum
charge = []          # List of lists containing charge for each spectrum
freq = []            # List of lists containing frequencies for each spectrum
dfreq = []           # List of lists containing dfreq for each spectrum
pol_degree = []      # List containing polynomial degrees for each spectrum
delta_sq = []        # List of lists containing delta_sq for each spectrum
item = []            # List of lists containing identities for each spectrum
identification = []  # List containing the identification values
benergy = []         # List of lists containing benergy for each spectrum
masses = np.zeros((I, 2))  # Masses array (You should define this)

# Function to fit data with a linear function
def linear_function(x, a, b):
    return a * x + b

# Function to set delta_sq
def set_delta_sq(iter):
    global I, S, max_pol_degree, masses, delta_sq

    delta_in = []

    if iter == 0:
        for s in range(S):
            mtoq = [mass[s][i] / charge[s][i] for i in range(len(mass[s]))]
            popt, _ = curve_fit(linear_function, freq[s], mtoq)
            par1 = popt[0] #here there was a 1, i think wrong
            for in_val in range(len(mass[s])):
                delta_in.append((par1 * dfreq[s][in_val]) ** 2)
            delta_sq.append(delta_in)
            delta_in = []

    else:
        for s in range(S):
            for in_val in range(len(mass[s])):
                col = np.zeros((pol_degree[s] + 1, 1))
                for k in range(pol_degree[s] + 1):
                    st = 0
                    for jm in range(len(mass[s])):
                        for id in range(I):
                            if identification[id] == item[s][jm]:
                                st += (1 / delta_sq[s][jm]) * (masses[id, 0] + benergy[s][jm]) / charge[s][jm] * (freq[s][jm] ** k)
                                break
                    col[k, 0] = st

                as_values = np.dot(AI_matrices[s], col)
                dtmp = 0

                for k in range(1, pol_degree[s] + 1):
                    dtmp += k * as_values[k, 0] * (freq[s][in_val] ** (k - 1)) * dfreq[s][in_val]

                mass_in_sp = 0

                for k in range(pol_degree[s] + 1):
                    mass_in_sp += as_values[k, 0] * (freq[s][in_val] ** k)

                delta_in.append(dtmp ** 2)

            delta_sq[s] = delta_in
            delta_in = []

# Call the function to set delta_sq
set_delta_sq(0)  # Pass 0 for the first iteration, and 1 for subsequent iterations


In [ ]:
import numpy as np

# Assumed defined global variables
I = 0                # Number of identities (You should define this)
masses = np.zeros((I, 2))  # Masses array (You should define this)
masses_prev = np.zeros((I, 2))  # Masses array from previous iteration (You should define this)
ref = []             # List of references (You should define this)
identification = []  # List containing the identification values (You should define this)

# Function to compare the current and previous iteration masses
def compare_iter():
    global I, masses, masses_prev
    for i in range(I):
        diff = (masses[i, 0] - masses_prev[i, 0]) * 9.31494
        if diff > 1:
            return True
    return False

# Function to print the results
def print_results():
    global I, masses, ref, identification

    print("Identity     is ref(0) \t mass in u \t\t ame_mass in u   diff \n")
    for i in range(I):
        id_name = str(identification[i])
        z = int(id_name[0:2])
        a = int(id_name[2:5])
        q = z
        is_val = int(id_name[5:])
        
        with open(nuclide_table, "r") as table:
            for line in table:
                tA, tZ, tQ, tEl, ti, table_mass, tmdm, tex, tE, tEde, tThl = map(int, line.strip().split())
                if tA == a and tZ == z and tQ == z and ti == is_val:
                    print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref[i]:1d} \t {masses[i, 0] / 100000:.6f}   {masses[i, 1] * amu / 100000:.2f} \t {table_mass * z:.6f} {(masses[i, 0] / 100000 - table_mass * z) * amu:.2f}")
                    break

# Call the functions to compare the iterations and print results
if compare_iter():
    print_results()


In [ ]:
import numpy as np

# Assumed defined global variables
I = 0  # Number of identities (You should define this)
identification_backup = []  # List containing the initial identification values (You should define this)
ref_backup = []  # List containing the initial reference values (You should define this)
refs_values = np.zeros(I)  # Array to store the final mass values (You should define this)
refs_unc_values = np.zeros(I)  # Array to store the final mass uncertainties (You should define this)

# Function to find and print changed identities with updated references
def print_results_again(r):
    global I, refs_values, refs_unc_values, identification_backup, ref_backup

    found_changed = False
    for i in range(len(identification_backup)):
        ii = 0
        while ii < I:
            if identification_backup[i] == identification[ii] and ref_backup[i] != ref[ii]:
                print(f"{i} was changed")
                id_name = str(identification[ii])
                z = int(id_name[0:2])
                a = int(id_name[2:5])
                q = z
                is_val = int(id_name[5:])

                with open(nuclide_table, "r") as table:
                    for line in table:
                        tA, tZ, tQ, tEl, ti, table_mass, tmdm, tex, tE, tEde, tThl = map(int, line.strip().split())
                        if tA == a and tZ == z and tQ == z and ti == is_val:
                            if ref[ii] != 0:
                                if ref[ii] == -1:
                                    print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref_backup[i]:1d}")
                                else:
                                    print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref_backup[i]:1d} \t {refs_values[i]:10.6f}   {refs_unc_values[i]:6.2f} \t {(refs_values[i] - table_mass * z) * amu:10.2f}    {tmdm / 1000000 * amu:6.2f}")
                            else:
                                print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref[ii]:1d} \t {masses[ii, 0] / 100000:10.6f}   {masses[ii, 1] * 9.31494:6.2f} \t {(masses[ii, 0] / 100000 - table_mass * z) * amu:10.2f}    {tmdm / 1000000 * amu:6.2f}")
                            break
                refs_values[i] = masses[ii, 0] / 100000
                refs_unc_values[i] = masses[ii, 1] * amu / 100000
                found_changed = True
                break
            else:
                if identification_backup[i] != identification[ii]:
                    ii += 1
                else:
                    break

    if not found_changed:
        counter = 0
        for i in range(len(identification_backup)):
            if ref_backup[i] == 0:
                if counter == r:
                    refs_values[i] = -1
                    refs_unc_values[i] = -1
                    print(f"{i} was changed, but deleted due to extrapolation")
                    break
                counter += 1

# Function to print the final results
def print_results_final():
    global I, identification_backup, refs_values, refs_unc_values, ref_backup

    for i in range(len(identification_backup)):
        found_id = False
        id_name = str(identification_backup[i])
        z = int(id_name[0:2])
        a = int(id_name[2:5])
        q = z
        is_val = int(id_name[5:])
        ii = 0
        while ii < I:
            if identification_backup[i] == identification[ii]:
                with open(nuclide_table, "r") as table:
                    for line in table:
                        tA, tZ, tQ, tEl, ti, table_mass, tmdm, tex, tE, tEde, tThl = map(int, line.strip().split())
                        if tA == a and tZ == z and tQ == z and ti == is_val:
                            if refs_values[i] != 0:
                                if refs_values[i] == -1:
                                    print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref_backup[i]:1d}")
                                else:
                                    print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref_backup[i]:1d} \t {refs_values[i]:10.6f}   {refs_unc_values[i]:6.2f} \t {(refs_values[i] - table_mass * z) * amu:10.2f}    {tmdm / 1000000 * amu:6.2f}")
                            else:
                                print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref[ii]:1d} \t {masses[ii, 0] / 100000:10.6f}   {masses[ii, 1] * 9.31494:6.2f} \t {(masses[ii, 0] / 100000 - table_mass * z) * amu:10.2f}    {tmdm / 1000000 * amu:6.2f}")
                            break
                found_id = True
                break
            else:
                ii += 1
        if not found_id:
            print(f"{z:2d} {a:3d} {q:2d} {is_val:1d} \t {ref_backup[i]:1d}")

# Call the functions to print the results again and final results
print_results_again(r)  # You should specify the value of 'r'
print_results_final()


In [ ]:
def change_refs(id_change):
    global ref_backup, ref

    ref_backup = ref[:]
    counter = 0
    for i in range(len(ref)):
        if ref[i] == 0:
            if counter == id_change:
                ref[i] = 1
                print(f"{identification[i]} switched off")
                break
            counter += 1

def change_refs_back():
    global ref, ref_backup
    ref = ref_backup[:]

def backup_vectors():
    global mass_backup, dmass_backup, freq_backup, dfreq_backup, item_backup, charge_backup, benergy_backup, identification_backup, ref_backup, I_b

    mass_backup = mass[:]
    dmass_backup = dmass[:]
    freq_backup = freq[:]
    dfreq_backup = dfreq[:]
    item_backup = item[:]
    charge_backup = charge[:]
    benergy_backup = benergy[:]
    identification_backup = identification[:]
    ref_backup = ref[:]
    I_b = I

def check_extrapolation():
    global I, identification, ref

    fContinue = True
    while fContinue:
        fContinue = False
        for i in range(len(freq)):
            for k in range(I):
                if item[i][0] == identification[k]:
                    if ref[k] == 1:
                        freq[i].pop(0)
                        dfreq[i].pop(0)
                        charge[i].pop(0)
                        benergy[i].pop(0)
                        mass[i].pop(0)
                        dmass[i].pop(0)
                        item[i].pop(0)
                        fContinue = True
                    break

    fContinue = True
    while fContinue:
        fContinue = False
        for i in range(len(freq)):
            for k in range(I):
                if item[i][-1] == identification[k]:
                    if ref[k] == 1:
                        freq[i].pop(-1)
                        dfreq[i].pop(-1)
                        charge[i].pop(-1)
                        benergy[i].pop(-1)
                        mass[i].pop(-1)
                        dmass[i].pop(-1)
                        item[i].pop(-1)
                        fContinue = True
                    break

    i = 0
    while i < I:
        found = False
        for s in range(len(freq)):
            for n in range(len(freq[s])):
                if item[s][n] == identification[i]:
                    found = True
        if not found:
            identification.pop(i)
            ref.pop(i)
            I -= 1
            print(f"{i} deleted")
        else:
            i += 1


In [ ]:
def restore_vectors():
    global mass, dmass, freq, dfreq, item, charge, benergy, identification, ref, I

    mass = mass_backup[:]
    dmass = dmass_backup[:]
    freq = freq_backup[:]
    dfreq = dfreq_backup[:]
    item = item_backup[:]
    charge = charge_backup[:]
    benergy = benergy_backup[:]
    identification = identification_backup[:]
    ref = ref_backup[:]
    I = I_b


In [ ]:
import sys

def main():
    if len(sys.argv) > 1:
        db_name = sys.argv[1]
    else:
        print("1st. argument = database name. db contains f df z a q is que. For example: 44464.00 3.78 77 195 75 0 0. And a '0' on new line to indicate end of spectrum.")
        return 0

    fill_vectors()
    backup_vectors()
    # check_extrapolation()
    fContinue = True
    iter = 0
    while fContinue and iter < 10:
        set_delta_sq(iter)
        fill_matrix_A()
        fill_matrix_W()
        fContinue = compare_iter()
        iter += 1

    print_results()
    print("Number of spectra:", S)
    print("Number of identities:", I)
    print("Number of iterations:", iter)
    print("Number of references:", R)
    print("The W-matrix determinant is", W.Determinant(), "\n")
    if fContinue:
        print("Minimum has not been reached!")
    print("Number of spectra:", S)
    print("Number of identities:", I)

    # Here starts part, where you switch off each of references and recalculate everything again
    '''
    for r in range(R):
        delta_sq.clear()
        restore_vectors()
        change_refs(r)
        check_extrapolation()
        fContinue = True
        iter = 0
        while fContinue and iter < 10:
            set_delta_sq(iter)
            fill_matrix_A()
            fill_matrix_W()
            fContinue = compare_iter()
            iter += 1
        if fContinue:
            print("%%%%%%%%%%%%%%%%Minimum has not been reached!%%%%%%%%%%%%%%%%%")
        print_results_again(r)
        # change_refs_back()
        print("Number of spectra:", S)
        print("Number of identities:", I)
        print("Number of iterations:", iter)
        print("The W-matrix determinant is", W.Determinant(), "\n")
        print("----------------------------------------------")
    # change_refs_back()  # do I need this now?
    restore_vectors()
    check_extrapolation()
    delta_sq.clear()
    fContinue = True
    iter = 0
    while fContinue and iter < 10:
        set_delta_sq(iter)
        fill_matrix_A()
        fill_matrix_W()
        fContinue = compare_iter()
        iter += 1
    print_results_final()
    '''
    return 1

if __name__ == "__main__":
    main()
